# Setup Environment

In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline
from transformers.models.auto.modeling_tf_auto import TFAutoModelForSequenceClassification
from transformers.models.bart.modeling_tf_bart import TFBartForSequenceClassification
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import transformers
import torch
import datasets

/N/u/jmonser/Carbonate/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-03 15:23:14.223953: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /N/soft/rhel7/python/3.9.8/lib:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/compiler/lib/intel64:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/ipp/lib/intel64:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/compiler/lib/intel64_lin:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/mkl/lib/intel64_lin:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/tbb/lib/intel64/gcc4.7:/N/soft/rhel7/intel/19.

# Load Data

## Load Patent Data

In [1]:
!pwd

/N/slate/jmonser/dsip_ria


In [7]:
ds = datasets.load_dataset('csv', data_files='./patents_h2_2020.csv')

Extracting data files: 100%|██████████████████████| 1/1 [00:00<00:00, 69.96it/s]


Dataset csv downloaded and prepared to /N/u/jmonser/Carbonate/.cache/huggingface/datasets/csv/default-6f38d5e898048daf/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  4.53it/s]


In [6]:
# patent_df = pd.read_csv('patents_h2_2020.csv', usecols=['abstract', 'claim_1', 'patent_num'], index_col='patent_num')
# patent_df = patent_df.agg(' '.join, axis=1)
# patent_df.to_csv("patents_h2_2020.csv")

In [9]:
patent_df = pd.read_pickle('patent_df.pkl').iloc[:5]
patent_df.index = patent_df['patent_num']
text_cols = ['abstract', 'claim_1']
patent_df = patent_df[text_cols]
patent_df = patent_df.agg(' '.join, axis=1)
patent_df.head()

patent_num
10524402    A method of real-time plant selection and remo...
10524403    A method is provided for determining a weight ...
10524404    A plow bottom is configured to be easily reset...
10524405    An agricultural implement system includes a ro...
10524406    An agricultural hitch with a system for manage...
dtype: object

In [19]:
# patent_df.to_csv('patents.csv', index=False)

In [70]:
patent_df.iloc[0]

'A method of real-time plant selection and removal from a plant field including capturing a first image of a first section of the plant field, segmenting the first image into regions indicative of individual plants within the first section, selecting the optimal plants for retention from the first image based on the first image and the previously thinned plant field sections, sending instructions to the plant removal mechanism for removal of the plants corresponding to the unselected regions of the first image from the second section before the machine passes the unselected regions, and repeating the aforementioned steps for a second section of the plant field adjacent the first section in the direction of machine travel. Method and apparatus for automated plan necrosis'

In [7]:
patent_df.shape

(1583074,)

## Load NAICS Information

My first attempt will be to create a NAICS-2 classifier, and then create a separate model for each of the NAICS-2 codes to split down to NAICS-4 level.

To start, we will load the codes and names of the NAICS-2 labels.

In [2]:
filepath = './Project_Patent_Classification/Data/NAICS/2-6 digit_2022_Codes.xlsx'
naics = pd.read_excel(filepath)
naics = naics.iloc[:, :3]
naics.dropna(inplace=True, axis=0, how='all')
idx = naics.iloc[:,1].astype('string').str.match('^(\d{2}|\d{2}\-\d{2})$')
naics4 = naics[naics.iloc[:,1].astype('string').str.match('^\d{4}$')].iloc[:,-1].tolist()

In [12]:
naics[naics.iloc[:,1].astype('string').str.match('^312.{1}$')]

,Seq. No.,2022 NAICS US Code,2022 NAICS US Title
347,347.0,3121,Beverage Manufacturing
358,358.0,3122,Tobacco Manufacturing


In [4]:
naics4

['Oilseed and Grain Farming',
 'Vegetable and Melon Farming',
 'Fruit and Tree Nut Farming',
 'Greenhouse, Nursery, and Floriculture Production',
 'Other Crop Farming',
 'Cattle Ranching and Farming',
 'Hog and Pig Farming',
 'Poultry and Egg Production',
 'Sheep and Goat Farming',
 'Aquaculture',
 'Other Animal Production',
 'Timber Tract Operations',
 'Forest Nurseries and Gathering of Forest Products',
 'Logging',
 'Fishing',
 'Hunting and Trapping',
 'Support Activities for Crop Production',
 'Support Activities for Animal Production',
 'Support Activities for Forestry',
 'Oil and Gas Extraction',
 'Coal Mining',
 'Metal Ore Mining',
 'Nonmetallic Mineral Mining and Quarrying',
 'Support Activities for Mining',
 'Electric Power Generation, Transmission and Distribution',
 'Natural Gas Distribution ',
 'Water, Sewage and Other Systems ',
 'Residential Building Construction',
 'Nonresidential Building Construction',
 'Utility System Construction',
 'Land Subdivision',
 'Highway, Stre

In [11]:
len(naics4)

308

# Load BART model

In [11]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli",
#                       device='cuda:0'
                     )

In [51]:

list(ds['train'][:2].values())[0]

['A method of real-time plant selection and removal from a plant field including capturing a first image of a first section of the plant field, segmenting the first image into regions indicative of individual plants within the first section, selecting the optimal plants for retention from the first image based on the first image and the previously thinned plant field sections, sending instructions to the plant removal mechanism for removal of the plants corresponding to the unselected regions of the first image from the second section before the machine passes the unselected regions, and repeating the aforementioned steps for a second section of the plant field adjacent the first section in the direction of machine travel. Method and apparatus for automated plan necrosis',
 'A method is provided for determining a weight of a payload carried by a support structure of an agricultural utility vehicle via a hitch. The hitch includes at least one upper link and at least one lower link. The 

In [12]:
# naics2labels
# patent_df.iloc[0]
import time
start = time.time()
results = classifier(patent_df.iloc[0], naics4.tolist(), multi_label=True)
end = time.time()
print("*"*50)
min_processing = np.round((end-start)/60,10)
print('It took', min_processing, 'minutes to predict labels for 1 row.')
full_processing = min_processing * patent_df.shape[0] / 60
print('This means it will take', full_processing, 'hours to process full dataset in current environment')

**************************************************
It took 1.3736089786 minutes to predict labels for 1 row.
This means it will take 0.11446741488333334 hours to process full dataset in current environment


In [6]:
results

[{'sequence': 'A method of real-time plant selection and removal from a plant field including capturing a first image of a first section of the plant field, segmenting the first image into regions indicative of individual plants within the first section, selecting the optimal plants for retention from the first image based on the first image and the previously thinned plant field sections, sending instructions to the plant removal mechanism for removal of the plants corresponding to the unselected regions of the first image from the second section before the machine passes the unselected regions, and repeating the aforementioned steps for a second section of the plant field adjacent the first section in the direction of machine travel. Method and apparatus for automated plan necrosis',
  'labels': ['Management, Scientific, and Technical Consulting Services',
   'Commercial and Service Industry Machinery Manufacturing',
   'Industrial Machinery Manufacturing',
   'Computer and Periphera

Review with project sponsor: Are these scores accurate/desirable?